In [3]:
import pandas as pd
import folium

Координаты Московии 55°45′21″ с. ш. 37°37′04″ в. д.

In [4]:
 
moskow_map = folium.Map(location=[55.75, 37.61],
                        zoom_start=13)
moskow_map


In [5]:
df = pd.read_json(r'data-28509-2021-07-11.json', encoding='cp1251')
df10 = df[0:10]
df100 = df[0:100]

In [7]:
for row in df100.itertuples():
    latitude = row[1]
    longitude = row[2]
    folium.CircleMarker(location=[latitude, longitude],fill=True).add_to(moskow_map)
moskow_map
